In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version
!pip install pyspark

openjdk version "1.8.0_222"
OpenJDK Runtime Environment (build 1.8.0_222-8u222-b10-1ubuntu1~18.04.1-b10)
OpenJDK 64-Bit Server VM (build 25.222-b10, mixed mode)


In [0]:
from google.colab import files

In [0]:
files.upload()

Saving hacking.csv to hacking.csv


{'hacking.csv': b"Session_Connection_Time,Bytes Transferred,Kali_Trace_Used,Servers_Corrupted,Pages_Corrupted,Location,WPM_Typing_Speed\n8.0,391.09,1,2.96,7.0,Slovenia,72.37\n20.0,720.99,0,3.04,9.0,British Virgin Islands,69.08\n31.0,356.32,1,3.71,8.0,Tokelau,70.58\n2.0,228.08,1,2.48,8.0,Bolivia,70.8\n20.0,408.5,0,3.57,8.0,Iraq,71.28\n1.0,390.69,1,2.79,9.0,Marshall Islands,71.57\n18.0,342.97,1,5.1,7.0,Georgia,72.32\n22.0,101.61,1,3.03,7.0,Timor-Leste,72.03\n15.0,275.53,1,3.53,8.0,Palestinian Territory,70.17\n12.0,424.83,1,2.53,8.0,Bangladesh,69.99\n15.0,249.09,1,3.39,9.0,Northern Mariana Islands,70.77\n32.0,242.48,0,4.24,8.0,Zimbabwe,67.93\n23.0,514.54,0,3.18,8.0,Isle of Man,68.56\n9.0,284.77,0,3.12,9.0,Sao Tome and Principe,70.82\n27.0,779.25,1,2.37,8.0,Greece,72.73\n12.0,307.31,1,3.22,7.0,Solomon Islands,67.95\n21.0,355.94,1,2.0,7.0,Guinea-Bissau,72.0\n10.0,372.65,0,3.33,7.0,Burkina Faso,69.19\n20.0,347.23,1,2.33,7.0,Mongolia,70.41\n22.0,456.57,0,1.52,8.0,Nigeria,69.35\n25.0,582.03,0,

In [0]:
from pyspark.sql import SparkSession
from pyspark.ml.clustering import KMeans

In [0]:
spark=SparkSession.builder.appName('findHacker').getOrCreate()

In [0]:
spark

In [0]:
dataset=spark.read.csv('hacking.csv', header=True, inferSchema=True)

In [0]:
type(dataset)

pyspark.sql.dataframe.DataFrame

In [0]:
rdd_view=dataset.rdd

In [0]:
rdd_view

MapPartitionsRDD[12] at javaToPython at NativeMethodAccessorImpl.java:0

In [0]:
dataset.head

<bound method DataFrame.head of DataFrame[Session_Connection_Time: double, Bytes Transferred: double, Kali_Trace_Used: int, Servers_Corrupted: double, Pages_Corrupted: double, Location: string, WPM_Typing_Speed: double]>

In [0]:
dataset.describe().show()

+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|summary|Session_Connection_Time| Bytes Transferred|   Kali_Trace_Used|Servers_Corrupted|   Pages_Corrupted|   Location|  WPM_Typing_Speed|
+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|  count|                    334|               334|               334|              334|               334|        334|               334|
|   mean|     30.008982035928145| 607.2452694610777|0.5119760479041916|5.258502994011977|10.838323353293413|       null|57.342395209580864|
| stddev|     14.088200614636158|286.33593163576757|0.5006065264451406| 2.30190693339697|  3.06352633036022|       null| 13.41106336843464|
|    min|                    1.0|              10.0|                 0|              1.0|               6.0|Afghanistan|              40.0|
|    max|           

In [0]:
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler

In [0]:
feat_cols=['Session_Connection_Time', 'Bytes Transferred', 'Kali_Trace_Used', 'Servers_Corrupted', 'Pages_Corrupted', 'WPM_Typing_Speed']

In [0]:
[x in dataset.columns for x in feat_cols]

In [0]:
vec_assembler=VectorAssembler(inputCols=feat_cols, outputCol='features')

In [0]:
final_data=vec_assembler.transform(data)

In [0]:
final_data.show

In [0]:
from pyspark.ml feature import StandardScaler

In [0]:
scaler=StandardScaler(inputCol='features', outputCol='scaledFeatures', withStd=True, withMean=False)

In [0]:
scalerModel=scaler.fit(final_data)

In [0]:
cluster_final_data=scalerModel.transform(final_data)

In [0]:
cluster_final_data.show()

In [0]:
kmeans3=KMeans(featuresCol='scaledFeatures', k=3)

In [0]:
kmeans3=KMeans(featuresCol='scaledFeatures', k=2)

In [0]:
model_k3=kmeans3.fit(cluster_final_data)
model_k2=kmeans2.fit(cluster_final_data)

In [0]:
wssse_k3=model_k3.computeCost(cluster_final_data)
wssse_k2=model_k2.computeCost(cluster_final_data)
wssse_k3
wssse_k2

In [0]:
model_k3.transform(cluster_final_data).groupBy('prediction').count().show()

In [0]:
model_k2.transform(cluster_final_data).groupBy('prediction').count().show()#must be equal so they are two hackers and not 3 even if the sum of squared is higher

In [0]:
for k in range(2,9): 
  kmeans=KMeans(featuresCol='scaledFeatures', k=k
                model=kmeans.fit(cluster_final_data))